<a href="https://colab.research.google.com/github/pavithrajonna-93/Eurostat_Manufacturing_vs_Services_Analysis/blob/main/Copy_of_project_eurostat_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

/tmp/ipython-input-464266642.py:12: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


NameError: name 'final_df' is not defined

In [ ]:
import matplotlib
matplotlib.use('Agg')
from matplotlib.backends.backend_pdf import PdfPages

from IPython.core.display import publish_display_data
import pandas as pd
#load the files
manufacturing = pd.read_csv('/Manufacturing_Master - Manufacturing_Master.csv')
services = pd.read_csv('/Services_Master - Services_Master.csv')

#ass sector labels
manufacturing['sector'] = 'manufacturing'
services ['sector'] = 'services'

#combine into one final dataset
final_df = pd.concat([manufacturing, services], ignore_index = True)

#show firts rows to confirm
final_df.head()

#step 4
final_df.info()
final_df.describe()

#step 5
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize =(12,6))

# Clean column names by stripping whitespace
final_df.columns = final_df.columns.str.strip()

#step 6
final_df.info()
final_df.describe()
final_df.isna().sum()
final_df.duplicated().sum()

final_df.to_excel("final_growth_comparison.xlsx", index=False)

manufacturing = final_df[final_df['sector']=='manufacturing']
services = final_df[final_df['sector']=='services']

manufacturing.to_excel("manufacturing_final.xlsx", index = False)
services.to_excel("services_final.xlsx", index=False)

final_df = final_df.rename(columns = {'Volataility_Range': 'Volatility_Range'})

final_df.columns

with PdfPages('All_Charts.pdf') as pdf:
    #----Chart1----
    plt.figure(figsize = (10,5))
    sns.barplot(data=final_df, x='Country', y='Overall_growth_Percent', hue= 'sector')
    plt.xticks(rotation=45)
    plt.title('overall growth comparision: manufacturing vs services')
    pdf.savefig()  # - SAVE THIS PLOT TO THE PDF
    plt.close()

    #---Chart2---
    plt.figure(figsize = (10,5))
    sns.barplot(data=final_df, x='Country', y='Avg_growth_percent', hue='sector')
    plt.xticks(rotation=45)
    plt.title("Average Growth percent: manufacturing vs services")
    pdf.savefig()  # - Save to same PDF
    plt.close()

    #---Chart3---
    plt.figure(figsize = (10,5))
    sns.barplot(data=final_df, x='Country', y= 'Volatility_Range', hue='sector')
    plt.xticks(rotation=45)
    plt.title("Volatility Comparison")
    pdf.savefig()
    plt.close()

    #---Chart4---
    plt.figure(figsize = (10,5))
    sns.barplot(data=final_df, x='Country', y='Avg_value', hue='sector')
    plt.xticks(rotation = 45)
    plt.title("Average Value Comparison")
    pdf.savefig()
    plt.close()


    with pd.ExcelWriter("complete_project_output.xlsx") as Writer:
      final_df.to_excel(Writer, sheet_name = "Final_Combined", index=False)
      manufacturing.to_excel(Writer, sheet_name = "manufacturing",index=False)
      services.to_excel(Writer,sheet_name="services",index=False)
#0. Improt libraries--
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

#1. LOAD FINAL DATA
df = final_df.copy()

for col in df.columns:
  df[col] = pd.to_numeric(df[col], errors='ignore')
numeric_df = df.select_dtypes(include = ['float64', 'int64'])


#2. Summary statistics---

Summary_stats = numeric_df.describe()
Summary_stats

#3. Histogram for each numeric variable---

def plot_histogram(df):
  df.hist(figsize=(15, 10), bins=15)
  plt.suptitle("Histograms of Numeric Variables", fontsize = 14)
  plt.tight_layout(rect=[0,0,1,0.96])
  plt.show()

# The function call 'plot_histograms' had a typo, changing to 'plot_histogram'
plot_histogram(numeric_df)

#4. Boxplots for Outliers

plt.figure(figsize=(15, 8))
sns.boxplot(data=numeric_df)
plt.xticks(rotation = 45)
plt.title("Boxplot for Outlier Detection")
plt.tight_layout()
plt.show()


#6. Correlation Matrix +HeatMap
corr_matrix = numeric_df.corr()
plt.figure(figsize =(12,7))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap")
plt.tight_layout()
plt.show()

#7. Top countries by Turnover/Enterprises/ PROD Value
metrics = ["Avg_growth_percent", "Avg_value", "Overall_growth_Percent"]

metrics = [m for m in metrics if m in df.columns]

for metric in metrics:
  plt.figure(figsize =(12, 6))
  top_10 = df.sort_values(metric, ascending = False).head(10)
  sns.barplot(data=top_10, x='Country', y=metric)
  plt.xticks(rotation = 75)
  plt.title(f"Top 10 Countries by {metric}")
  plt.tight_layout()
  plt.show()

#8. EXPORT ALL CHARTS INTO ONE PDF

with PdfPages("EDA_CHARTS_ALL.pdf") as pdf:

  #summary stats table - save as figure
  fig, ax=plt.subplots(figsize =(12,4))
  ax.axis("tight")
  ax.axis("off")
  ax.table(cellText=Summary_stats.values,
           colLabels=Summary_stats.columns,
           rowLabels=Summary_stats.index,
           loc="center")
  pdf.savefig(fig)
  plt.close()

  #Histograms
  df.hist(figsize=(15,10), bins=15)
  plt.suptitle("Histograms of Numeric Variables", fontsize = 14)
  plt.tight_layout()
  pdf.savefig()
  plt.close()

  #Heatmap
  plt.figure(figsize=(12,7))
  sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
  plt.title("Correlation Heatmap")
  plt.tight_layout()
  pdf.savefig()
  plt.close()

  #top_10 Metric charts
  for metric in metrics:
    plt.figure(figsize=(12,6))
    top_10 = df.sort_values(metric, ascending=False).head(10)
    sns.barplot(data=top_10, x='Country', y=metric)
    plt.xticks(rotation=75)
    plt.title(f"Top 10 Countries by {metric}")
    plt.tight_layout()
    pdf.savefig()
    plt.close()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Country                 36 non-null     object 
 1   Category                36 non-null     object 
 2   Avg_growth_percent      36 non-null     float64
 3   Avg_value               36 non-null     float64
 4   Volataility_Range       36 non-null     float64
 5   Overall_growth_Percent  36 non-null     float64
 6   sector                  36 non-null     object 
dtypes: float64(4), object(3)
memory usage: 2.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Country                 36 non-null     object 
 1   Category                36 non-null     object 
 2   Avg_growth_percent      36 non-null     float64
 3

/tmp/ipython-input-3815024726.py:100: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
